In [ ]:
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install dgl
!pip install rdkit
!pip install deepchem
!pip install gensim
!pip install git+https://github.com/samoturk/mol2vec
!pip install transformers

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-jtk6nqln
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec /tmp/pip-req-build-jtk6nqln
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import random
import os.path as osp
from rdkit import Chem
import deepchem as dc
from deepchem.feat.smiles_tokenizer import BasicSmilesTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling, convert, to_dense_adj
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.nn.conv import MessagePassing

import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR,MultiplicativeLR
from torch import Tensor
from torch.utils.data import DataLoader

from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle

from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec
import torch
from sklearn.metrics import roc_auc_score ,auc,precision_recall_curve,f1_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/libpyg.so: undefined symbol: _ZN2at4_ops10zeros_like4callERKNS_6TensorEN3c108optionalINS5_10ScalarTypeEEENS6_INS5_6LayoutEEENS6_INS5_6DeviceEEENS6_IbEENS6_INS5_12MemoryFormatEEE
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


In [ ]:
DDI_graph = pd.read_csv('https://raw.githubusercontent.com/liiniix/BioSNAP/master/ChCh-Miner/ChCh-Miner_durgbank-chem-chem.tsv', sep='\t')
#DDI_graph = pd.read_csv('https://raw.githubusercontent.com/sshaghayeghs/molSMILES/main/drugbankDDI.csv', sep=',')

#save the raw data as our own data so that we do not depend on other apis.
DDI_graph.rename(columns={'Drug1': 'src', 'Drug2': 'dst'}, inplace=True)
DrugIDs_in_graph = np.unique(DDI_graph.values) # there are 1514 unique drugs in the graph

DDI_graph

,src,dst
0,DB00862,DB00966
1,DB00575,DB00806
2,DB01242,DB08893
3,DB01151,DB08883
4,DB01235,DB01275
...,...,...
48509,DB00542,DB01354
48510,DB00476,DB01239
48511,DB00621,DB01120
48512,DB00808,DB01356


In [ ]:
drugsSMILES = pd.read_csv('https://raw.githubusercontent.com/sshaghayeghs/molSMILES/main/structure%20links%202.csv')
drugID_smiles = drugsSMILES[["DrugBank ID", "SMILES"]]
drugID_smiles.dropna(inplace=True)
drugID_smiles.reset_index(drop=True, inplace=True)

<ipython-input-5-26bacc5df7ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drugID_smiles.dropna(inplace=True)


In [ ]:
drugsDESC = pd.read_csv('https://raw.githubusercontent.com/sshaghayeghs/molSMILES/main/Drug_description.csv')
drugID_DESC = drugsDESC[["Drug ID", "Discription"]]
drugID_DESC.dropna(inplace=True)
drugID_DESC.reset_index(drop=True, inplace=True)

<ipython-input-6-c2147157fec6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drugID_DESC.dropna(inplace=True)


In [ ]:
drugID_DESC

,Drug ID,Discription
0,DB00001,Lepirudin is a recombinant hirudin formed by 6...
1,DB00002,Cetuximab is a recombinant chimeric human/mous...
2,DB00003,Dornase alfa is a biosynthetic form of human d...
3,DB00004,A recombinant DNA-derived cytotoxic protein co...
4,DB00005,Dimeric fusion protein consisting of the extra...
...,...,...
15230,DB17382,AUM-601 is a highly selective pan-TRK(tropomyo...
15231,DB17383,;;;;
15232,DB17384,;;;;
15233,DB17385,;;;;


In [ ]:
def is_valid_molecule(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False


In [ ]:
valid_smiles = pd.DataFrame(drugID_smiles)
valid_smiles['IsValidMolecule'] = drugID_smiles['SMILES'].apply(is_valid_molecule)
df_valid_molecules = valid_smiles[valid_smiles['IsValidMolecule']]

# Drop the temporary 'IsValidMolecule' column
df_valid_molecules = df_valid_molecules.drop(columns=['IsValidMolecule'])

[17:09:24] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[17:09:24] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[17:09:24] SMILES Parse Error: check for mistakes around position 84:
[17:09:24] C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O
[17:09:24] ~~~~~~~~~~~~~~~~~~~~^
[17:09:24] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]'
[17:09:24] Explicit valence for atom # 19 O, 2, is greater than permitted
[17:09:24] Explicit valence for atom # 0 O, 3, is greater than perm

In [ ]:
drugID_smiles_ddi = drugID_smiles[drugID_smiles['DrugBank ID'].isin(list(np.unique(DDI_graph.values)))]
drugID_smiles_ddi=drugID_smiles_ddi.reset_index(drop=True)
drugID_DESC_ddi = drugID_DESC[drugID_DESC['Drug ID'].isin(list(np.unique(DDI_graph.values)))]
drugID_DESC_ddi=drugID_DESC_ddi.reset_index(drop=True)

In [ ]:
drugID_DESC_ddi

,Drug ID,Discription
0,DB00005,Dimeric fusion protein consisting of the extra...
1,DB00006,Bivalirudin is a synthetic 20 residue peptide ...
2,DB00007,Leuprolide is a synthetic 9-residue peptide an...
3,DB00008,Peginterferon alfa-2a is a form of recombinant...
4,DB00009,Alteplase is a recombinant tissue plasminogen ...
...,...,...
1492,DB11248,Zinc gluconate is a zinc salt of gluconic acid...
1493,DB11256,Levomefolic acid (INN) is the metabolite of fo...
1494,DB11315,Methscopolamine is a quaternary ammonium deriv...
1495,DB00873,;;;;


In [ ]:
drugID_smiles_ddi

,DrugBank ID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,DB00050,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
...,...,...
1327,DB11248,[Zn++].[H][C@@](O)(CO)[C@@]([H])(O)[C@]([H])(O...
1328,DB11256,CN1[C@@H](CNC2=CC=C(C=C2)C(=O)N[C@@H](CCC(O)=O...
1329,DB11315,C[N+]1(C)[C@H]2C[C@@H](C[C@@H]1[C@H]1O[C@@H]21...
1330,DB00873,[H][C@@]12CC[C@](O)(C(=O)OCCl)[C@@]1(C)C[C@H](...


In [ ]:
drug_with_desc_smile = pd.merge(drugID_smiles_ddi, drugID_DESC_ddi, left_on='DrugBank ID', right_on='Drug ID', how='inner')

In [ ]:
len(drug_with_desc_smile)

1332

In [ ]:
def filter_desc(x):
  return len(x) > 20

In [ ]:
df_filtered = drug_with_desc_smile[drug_with_desc_smile['Discription'].apply(filter_desc)]

In [ ]:
len(df_filtered)

1325

In [ ]:
df_filtered = df_filtered.rename(columns={'Discription': 'Description'})


In [ ]:
df_filtered = df_filtered.rename(columns={'Drug ID': 'drug_id'})

In [ ]:
df_filtered

,DrugBank ID,SMILES,drug_id,Description
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,DB00006,Bivalirudin is a synthetic 20 residue peptide ...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,DB00007,Leuprolide is a synthetic 9-residue peptide an...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,DB00014,"Goserelin is a synthetic hormone. In men, it s..."
3,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,DB00035,"Desmopressin (dDAVP), a synthetic analogue of ..."
4,DB00050,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,DB00050,Cetrorelix is a man-made hormone that blocks t...
...,...,...,...,...
1326,DB11181,[H][C@]12CC[C@]([H])(C[C@@H](C1)OC(=O)C(O)C1=C...,DB11181,Homatropine is an anticholinergic drug that ac...
1327,DB11248,[Zn++].[H][C@@](O)(CO)[C@@]([H])(O)[C@]([H])(O...,DB11248,Zinc gluconate is a zinc salt of gluconic acid...
1328,DB11256,CN1[C@@H](CNC2=CC=C(C=C2)C(=O)N[C@@H](CCC(O)=O...,DB11256,Levomefolic acid (INN) is the metabolite of fo...
1329,DB11315,C[N+]1(C)[C@H]2C[C@@H](C[C@@H]1[C@H]1O[C@@H]21...,DB11315,Methscopolamine is a quaternary ammonium deriv...


In [ ]:
allowed_drugs = set(df_filtered['drug_id'].tolist())

In [ ]:
DDI_graph = DDI_graph[DDI_graph['src'].isin(allowed_drugs)]
DDI_graph = DDI_graph[DDI_graph['dst'].isin(allowed_drugs)]

In [ ]:
DDI_graph=DDI_graph.reset_index(drop=True)


In [ ]:
G = nx.from_pandas_edgelist(DDI_graph, 'src', 'dst')


In [ ]:
# Find the connected components
connected_components = nx.connected_components(G)

for component in connected_components:
    subgraph = G.subgraph(component)
    print("Subgraph Nodes:", subgraph.nodes())
    print("Subgraph Edges:", subgraph.edges())
    print("Subgraph Length:", len(subgraph))
    print()

Subgraph Nodes: ['DB00862', 'DB00966', 'DB00575', 'DB00806', 'DB01242', 'DB08893', 'DB01151', 'DB08883', 'DB01235', 'DB01275', 'DB00201', 'DB01609', 'DB00712', 'DB01418', 'DB01576', 'DB04842', 'DB06212', 'DB01232', 'DB09291', 'DB00382', 'DB06153', 'DB01017', 'DB01142', 'DB00214', 'DB01174', 'DB01222', 'DB09063', 'DB00898', 'DB01589', 'DB00762', 'DB09054', 'DB00780', 'DB01158', 'DB00186', 'DB09071', 'DB00484', 'DB00881', 'DB00396', 'DB00682', 'DB00368', 'DB01551', 'DB01320', 'DB09061', 'DB01167', 'DB00572', 'DB00921', 'DB09272', 'DB01026', 'DB08865', 'DB00999', 'DB01114', 'DB09042', 'DB00673', 'DB08918', 'DB01215', 'DB01100', 'DB06201', 'DB01095', 'DB01393', 'DB09072', 'DB00983', 'DB01223', 'DB00865', 'DB01623', 'DB00211', 'DB00316', 'DB00422', 'DB00876', 'DB00366', 'DB01189', 'DB00997', 'DB04905', 'DB00177', 'DB09401', 'DB01211', 'DB00603', 'DB01268', 'DB01394', 'DB00623', 'DB00656', 'DB00834', 'DB09039', 'DB00675', 'DB05812', 'DB00215', 'DB00420', 'DB00401', 'DB09059', 'DB00625', 'DB0

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
is_undirected = G.is_directed() is False
average_node_degree = sum(dict(G.degree()).values()) / num_nodes

has_isolated_nodes = any(deg == 0 for _, deg in G.degree())
has_self_loops = any(G.has_edge(n, n) for n in G.nodes())

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Is undirected: {is_undirected}")
print(f"Average node degree: {average_node_degree}")
print(f"Has isolated nodes: {has_isolated_nodes}")
print(f"Has self-loops: {has_self_loops}")


Number of nodes: 1316
Number of edges: 41528
Is undirected: True
Average node degree: 63.11246200607903
Has isolated nodes: False
Has self-loops: False


In [ ]:
num_samples = 100

# Sample positive edges (edges that exist in G)
positive_edges = list(G.edges())
sampled_positive = random.sample(positive_edges, min(num_samples, len(positive_edges)))

# Sample negative edges (edges that do NOT exist in G)
# networkx provides an iterator for non-edges:
non_edges = list(nx.non_edges(G))
sampled_negative = random.sample(non_edges, min(num_samples, len(non_edges)))

print("Positive edges sample:", sampled_positive)
print("Negative edges sample:", sampled_negative)

Positive edges sample: [('DB00535', 'DB09268'), ('DB01354', 'DB00949'), ('DB00503', 'DB01244'), ('DB00683', 'DB01264'), ('DB00542', 'DB01021'), ('DB01609', 'DB01132'), ('DB08820', 'DB00696'), ('DB00898', 'DB00950'), ('DB00470', 'DB04868'), ('DB09042', 'DB00450'), ('DB09242', 'DB00678'), ('DB00358', 'DB00832'), ('DB01403', 'DB00370'), ('DB00486', 'DB01440'), ('DB01238', 'DB00745'), ('DB00204', 'DB00199'), ('DB00557', 'DB00897'), ('DB08912', 'DB00317'), ('DB00956', 'DB00231'), ('DB00798', 'DB00948'), ('DB00571', 'DB09236'), ('DB01224', 'DB00794'), ('DB00366', 'DB00395'), ('DB01224', 'DB06203'), ('DB00540', 'DB00488'), ('DB09054', 'DB08881'), ('DB00503', 'DB00514'), ('DB08816', 'DB00686'), ('DB00393', 'DB00349'), ('DB00661', 'DB08815'), ('DB00790', 'DB00594'), ('DB00268', 'DB01351'), ('DB00912', 'DB00880'), ('DB01242', 'DB01104'), ('DB01418', 'DB00535'), ('DB00598', 'DB06714'), ('DB00543', 'DB01614'), ('DB02659', 'DB00658'), ('DB00668', 'DB06706'), ('DB09117', 'DB00854'), ('DB06218', 'DB0

In [ ]:
desc_and_smiles = dict()
drug_ids = df_filtered['drug_id'].tolist()
descriptions = df_filtered['Description'].tolist()
smiles = df_filtered['SMILES'].tolist()

for i, drug_id in enumerate(drug_ids):
  desc_and_smiles[drug_id] = (descriptions[i], smiles[i])

In [ ]:
df_pos_dict = {'drug1':[], 'desc1': [], 'smile1':[], 'drug2': [], 'desc2': [], 'smile2': []}

In [ ]:
df_neg_dict = {'drug1':[], 'desc1': [], 'smile1':[], 'drug2': [], 'desc2': [], 'smile2': []}

In [ ]:
for d1, d2 in sampled_positive:
  df_pos_dict['drug1'].append(d1)
  df_pos_dict['drug2'].append(d2)
  df_pos_dict['desc1'].append(desc_and_smiles[d1][0])
  df_pos_dict['desc2'].append(desc_and_smiles[d2][0])
  df_pos_dict['smile1'].append(desc_and_smiles[d1][1])
  df_pos_dict['smile2'].append(desc_and_smiles[d2][1])

In [ ]:
for d1, d2 in sampled_negative:
  df_neg_dict['drug1'].append(d1)
  df_neg_dict['drug2'].append(d2)
  df_neg_dict['desc1'].append(desc_and_smiles[d1][0])
  df_neg_dict['desc2'].append(desc_and_smiles[d2][0])
  df_neg_dict['smile1'].append(desc_and_smiles[d1][1])
  df_neg_dict['smile2'].append(desc_and_smiles[d2][1])

In [ ]:
df_neg = pd.DataFrame.from_dict(df_neg_dict)

In [ ]:
df_pos = pd.DataFrame.from_dict(df_pos_dict)

In [ ]:
df_pos.to_csv('ddi_positive_example.csv')
df_neg.to_csv('ddi_negative_example.csv')

In [ ]:
transform = RandomLinkSplit(num_val=0.2,
    num_test=0.2,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0)

In [ ]:
def no_feature(smiles,DDI_graph):
  #DrugIDs_in_graph = np.unique(DDI_graph.values)
  features = np.ones((len(smiles),100))
  print('no_feature')
  return features,DDI_graph


In [ ]:
def Morgan(smiles,DDI_graph):
  featurizer = dc.feat.CircularFingerprint(size=100, radius=1)
  dataset=smiles['SMILES']
  features = pd.DataFrame(columns = [i for i in range(100)])
  for i in range(len(dataset)):
    features.loc[i] = featurizer.featurize(dataset[i])[0]
  print('Morgan')
  return features.values,DDI_graph

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
    def encode(self, x, edge_index):
        x=self.conv1(x, edge_index)
        x=F.dropout(x, p=0.3)
        x=F.relu(x)
        x=self.conv2(x, edge_index)
        x=F.dropout(x, p=0.3)
        x=F.relu(x)
        x=self.conv3(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()

    z = model.encode(train_data.x, train_data.edge_index) # initializing GCN model
    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    roc=roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    label=data.edge_label.cpu().numpy()
    score=out.cpu().numpy()
    return roc,label,score

In [ ]:
Embedding_models={'no_feature':no_feature(drugID_smiles_ddi,DDI_graph),
                  'Morgan':Morgan(drugID_smiles_ddi,DDI_graph)}

no_feature


[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerator
[17:09:31] DEPRECATION WARNING: please use MorganGenerat

Morgan


[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerator
[17:09:36] DEPRECATION WARNING: please use MorganGenerat

In [ ]:
lmbda = lambda epoch: 0.96


In [ ]:
def PyG_data(feature,DDI_graph):
  DrugIDs_in_graph = np.unique(DDI_graph.values)
  node_id_map = {node_name: i for i, node_name in enumerate(DrugIDs_in_graph)}
  # Replace node names with integer IDs in the edge list
  src = [node_id_map[node_name] for node_name in DDI_graph['src']]
  dst = [node_id_map[node_name] for node_name in DDI_graph['dst']]
  # Stack the arrays side by side to create a 2D array
  combined_array = np.column_stack((np.array(src), np.array(dst)))
  edge_index = []  # List of tuples representing edges between drugs
  for drug_1, drug_2 in combined_array:
    # Create an undirected graph by adding edges in both directions
    edge_index.append((drug_1, drug_2))
    edge_index.append((drug_2, drug_1))
  #Replace node names with integer IDs in the feature
  feature=torch.tensor(feature,dtype=torch.float32)
  data = Data(x=feature, edge_index=torch.tensor(edge_index).t().contiguous())
  return data

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
print(device)

cuda


In [ ]:
LR=[0.0003]
modelname=['no_feature','Morgan']
AUC=pd.DataFrame(columns = ['0.0003'])
PR=pd.DataFrame(columns = ['0.0003'])
AUC['Embedding']=modelname
PR['Embedding']=modelname

best_scores_calculated = list()

for l in LR:
  results_AUC=pd.DataFrame()
  results_PR=pd.DataFrame()
  for modelname, emb in Embedding_models.items():
    print('-------------------------------')
    print('=========',modelname,'=========')
    print('-------------------------------')
    data=PyG_data(emb[0],emb[1])
    train_data, val_data, test_data = transform(data)
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    test_data = test_data.to(device)
    model = Net(data.num_features, 256, 256).to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=l)
    scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)
    criterion = torch.nn.BCEWithLogitsLoss()
    neg_edge_index = negative_sampling(
          edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
          num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
          [train_data.edge_label_index, neg_edge_index],
          dim=-1,
      )
    edge_label = torch.cat([
          train_data.edge_label,
          train_data.edge_label.new_zeros(neg_edge_index.size(1))
      ], dim=0)
    best_val_auc = final_test_auc = 0
    for epoch in range(1, 100):
        loss = train()
        val_auc = test(val_data)[0]
        test_auc = test(test_data)[0]
        label=test(test_data)[1]
        score=test(test_data)[2]
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            final_test_auc = test_auc
            best_scores=score
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}')

    precision, recall, thresholds = precision_recall_curve(label, best_scores)
    best_scores_calculated.append((label, best_scores))
    pr=auc(recall, precision)
    new_auc_row = pd.DataFrame({"Embedding": modelname, "AUC": final_test_auc}, index=[0])
    results_AUC = pd.concat([results_AUC, new_auc_row], ignore_index=True)

    # Replace results_PR.append with pd.concat
    new_pr_row = pd.DataFrame({"Embedding": modelname, "PR_AUC": pr}, index=[0])
    results_PR = pd.concat([results_PR, new_pr_row], ignore_index=True)
    del data

  AUC[str(l)]=results_AUC['AUC']
  PR[str(l)]=results_PR['PR_AUC']

-------------------------------
========= no_feature =========
-------------------------------
Epoch: 001, Loss: 11.4674, Val: 0.7590
Epoch: 002, Loss: 8.7114, Val: 0.8100
Epoch: 003, Loss: 6.6079, Val: 0.8337
Epoch: 004, Loss: 5.1679, Val: 0.8494
Epoch: 005, Loss: 4.0564, Val: 0.8477
Epoch: 006, Loss: 3.0355, Val: 0.8512
Epoch: 007, Loss: 2.4002, Val: 0.8483
Epoch: 008, Loss: 1.9020, Val: 0.8490
Epoch: 009, Loss: 1.5975, Val: 0.8505
Epoch: 010, Loss: 1.3314, Val: 0.8536
Epoch: 011, Loss: 1.1906, Val: 0.8527
Epoch: 012, Loss: 1.0514, Val: 0.8512
Epoch: 013, Loss: 0.9672, Val: 0.8528
Epoch: 014, Loss: 0.8939, Val: 0.8535
Epoch: 015, Loss: 0.8349, Val: 0.8543
Epoch: 016, Loss: 0.7949, Val: 0.8529
Epoch: 017, Loss: 0.7540, Val: 0.8520
Epoch: 018, Loss: 0.7252, Val: 0.8530
Epoch: 019, Loss: 0.7134, Val: 0.8547
Epoch: 020, Loss: 0.6978, Val: 0.8566
Epoch: 021, Loss: 0.6725, Val: 0.8570
Epoch: 022, Loss: 0.6696, Val: 0.8560
Epoch: 023, Loss: 0.6625, Val: 0.8569
Epoch: 024, Loss: 0.6555, Val:

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, accuracy_score, f1_score


def metric(y_true,y_score, threshold=0.7):
  # y_true = data.edge_label.cpu().numpy()
  # y_score = out.cpu().numpy()

  # Compute AUC (if needed)

  # Apply threshold to generate binary predictions
  y_pred = (y_score >= threshold).astype(int)

  # Compute confusion matrix and extract values
  cm = confusion_matrix(y_true, y_pred)
  TN, FP, FN, TP = cm.ravel()

  # Calculate metrics
  sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0.0  # equivalent to recall for positive class
  npv = TN / (TN + FN) if (TN + FN) != 0 else 0.0
  recall = recall_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  accuracy = accuracy_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  specificity = TN / (TN + FP) if (TN + FP) != 0 else 0.0

  # Return all metrics and raw values if needed
  return {
      "sensitivity": sensitivity,
      "npv": npv,
      "recall": recall,
      "precision": precision,
      "accuracy": accuracy,
      "specificity": specificity,
      "f1_score": f1,
      "labels": y_true,
      "scores": y_score
  }

In [ ]:
print(len(best_scores_calculated))

2


In [ ]:
metric(best_scores_calculated[0][0],best_scores_calculated[0][1])

{'sensitivity': 0.4114388922335942,
 'npv': 0.6173776908023484,
 'recall': 0.4114388922335942,
 'precision': 0.8910039113428944,
 'accuracy': 0.6805538832028898,
 'specificity': 0.9496688741721855,
 'f1_score': 0.5629324546952224,
 'labels': array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 'scores': array([0.66527563, 0.5605111 , 0.67852163, ..., 0.55350167, 0.6028616 ,
        0.51765496], dtype=float32)}

In [ ]:
metric(best_scores_calculated[1][0],best_scores_calculated[1][1])

{'sensitivity': 0.5092113184828416,
 'npv': 0.6541659596131003,
 'recall': 0.5092113184828416,
 'precision': 0.8766583747927031,
 'accuracy': 0.7187838651414811,
 'specificity': 0.9283564118001204,
 'f1_score': 0.6442227130779191,
 'labels': array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 'scores': array([0.7028271 , 0.65767974, 0.63228893, ..., 0.5289983 , 0.50544137,
        0.5210887 ], dtype=float32)}